<a href="https://colab.research.google.com/github/gacerioni/redis-workshop-json-search-vs/blob/master/redis-workshop-vector-similarity-search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Workshop - Redis como VectorDB - VSS e LLM (soon)

![Redis](https://redis.io/wp-content/uploads/2024/04/Logotype.svg?auto=webp&quality=85,75&width=120)


Bem-vind[ao]s ao Workshop! Vamos ter uma experiência hands-on sobre alguns temas centrais do Redis, bem além do Caching.


Para uma experiência premium, como a que eu quero que vocês tenham, recomendo MUITO utilizar o Redis Insight (App ou Web) pra apoiar na visualização dos dados.

https://redis.com/redis-enterprise/redis-insight/

In [ ]:
# Instale o Redis client e tambem o Hugging Face sentence transformers, pois vamos gerar os vetores aqui mesmo
!pip install -q redis sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.0/252.0 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 16.3 MB/s eta 0:00:00


Instale o Redis Stack localmente, só pra garantir!\
*(vai que você não está no Redis Cloud... quero que participes do lab sem Toil)*

In [ ]:
%%sh
curl -fsSL https://packages.redis.io/gpg | sudo gpg --dearmor -o /usr/share/keyrings/redis-archive-keyring.gpg
echo "deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb $(lsb_release -cs) main" | sudo tee /etc/apt/sources.list.d/redis.list
sudo apt-get update  > /dev/null 2>&1
sudo apt-get install redis-stack-server  > /dev/null 2>&1
redis-stack-server --daemonize yes


deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb jammy main
Starting redis-stack-server, database path /var/lib/redis-stack


### Conectando com o Redis server

In [ ]:
import os

REDIS_HOST = os.getenv("REDIS_HOST", "localhost")
REDIS_PORT = os.getenv("REDIS_PORT", "6379")
REDIS_PASSWORD = os.getenv("REDIS_PASSWORD", "")
#Replace values above with your own if using Redis Cloud instance
#REDIS_HOST="redis-18374.c253.us-central1-1.gce.cloud.redislabs.com"
#REDIS_PORT=18374
#REDIS_PASSWORD="1TNxTEdYRDgIDKM2gDfasupCADXXXX"

#shortcut for redis-cli $REDIS_CONN command
# If SSL is enabled on the endpoint add --tls
if REDIS_PASSWORD!="":
  os.environ["REDIS_CONN"]=f"-h {REDIS_HOST} -p {REDIS_PORT} -a {REDIS_PASSWORD} --no-auth-warning"
else:
  os.environ["REDIS_CONN"]=f"-h {REDIS_HOST} -p {REDIS_PORT}"

# If SSL is enabled on the endpoint, use rediss:// as the URL prefix
REDIS_URL = f"redis://:{REDIS_PASSWORD}@{REDIS_HOST}:{REDIS_PORT}"
INDEX_NAME = f"qna:idx"

# Test Redis connection
!redis-cli $REDIS_CONN PING

PONG


In [ ]:
import redis
redis = redis.Redis(
  host=REDIS_HOST,
  port=REDIS_PORT,
  password=REDIS_PASSWORD)
redis.ping()

True

In [ ]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from tqdm.auto import tqdm
from redis import Redis
from redis.commands.search.field import (
    NumericField,
    TagField,
    TextField,
    VectorField,
)
from redis.commands.search.indexDefinition import IndexDefinition, IndexType
from redis.commands.search.query import Query


tqdm.pandas()



### Embedding generation model

Aqui começa o samba. Vamos usar o `sentence-transformers/all-MiniLM-L6-v2` da HuggingFace. https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2 para gerar nosso embedding de vetores. Vai ser bacana, e vai abrir a mente sobre usos fora do mundo de Machine Learning... como buscas performáticas por features e aproximações semânticas.

Este modelo possui 384 dimensões, o que já é mais do que suficiente para a nossa demo, considerando os tokens e tudo mais.



In [ ]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

#### Download 12k+ tweets

Vamos usar um material do Anton, nosso gênio de outra Region, para subir 12k de Tweets pro Redis. Baixe o csv aqui mesmo:

In [ ]:
!wget https://raw.githubusercontent.com/antonum/Redis-VSS-Streamlit/main/Labelled_Tweets.csv

--2024-05-13 20:00:25--  https://raw.githubusercontent.com/antonum/Redis-VSS-Streamlit/main/Labelled_Tweets.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2486081 (2.4M) [text/plain]
Saving to: ‘Labelled_Tweets.csv’

Labelled_Tweets.csv 100%[===================>]   2.37M  --.-KB/s    in 0.01s   

2024-05-13 20:00:25 (233 MB/s) - ‘Labelled_Tweets.csv’ saved [2486081/2486081]



In [ ]:
df = pd.read_csv('Labelled_Tweets.csv').drop(columns=['created_at','score'])
# opcional, pra gente segurar o reggae caso estejamos travados no cap de 30MB do Free Tier.
#df=df.head(3000) #trim dataframe to fit results into 30MB Redis database
df


,id,full_text
0,1,@KennyDegu very very little volume. With $10T ...
1,2,#ES_F achieved Target 2780 closing above 50% #...
2,3,RT @KimbleCharting: Silver/Gold indicator crea...
3,4,@Issaquahfunds Hedged our $MSFT position into ...
4,5,RT @zipillinois: 3 Surprisingly Controversial ...
...,...,...
12415,12587,RT @PeterLBrandt: $SPX $ES_F \r\nFollowing thi...
12416,12588,RT @vieiraUAE: Fearless Alex Vieira Calls Best...
12417,12589,$spy $spx $qqq $ndx #nyse going from poking th...
12418,12590,RT @DavidScottAdams: On watch tomorrow // Pt. ...


### Gerando os Embeddings (o Vetor em si, finalmente)

Vamos gerar o "vector embedding" para o nosso dataframe.
Este passo pode levar 2-4 minutinhos, pois a GPU aqui é de demo rsrs


Notem que eu optei por utilizar o campo `full_text` para gerar os embeddings. Entretanto, em breve você irá ficar craque em decidir o que usar para gerar essa "impressão digital" da Entidade. Um filme, por exemplo, poderia usar uma concatenação do sumário, gênero, resenhas, paleta de cores, etc.


In [ ]:
def text_to_embedding(text):
  return model.encode(text).astype(np.float32).tobytes()

#generate vector embeddings
df["text_embedding"] = df["full_text"].progress_apply(text_to_embedding)
df.head()

  0%|          | 0/12420 [00:00<?, ?it/s]

,id,full_text,text_embedding
0,1,@KennyDegu very very little volume. With $10T ...,b'*\x92\x81\xbd\x14h\x8b\xbdy\xdf\xe4\xbc\xc0\...
1,2,#ES_F achieved Target 2780 closing above 50% #...,b'Q\x1b\x02\xbd\x14~/\xbd\xa6z\xb1\xbcb\x99\xd...
2,3,RT @KimbleCharting: Silver/Gold indicator crea...,b'\x10\xaa\xa3\xbd]}\x10\xbd\xc3\xe8\xb9=%\x08...
3,4,@Issaquahfunds Hedged our $MSFT position into ...,b'\xc4\x7f\xd1\xbc\xbe\n`\xbd79 =\xe4\xc0\xef=...
4,5,RT @zipillinois: 3 Surprisingly Controversial ...,b'\xc6\r\x1e\xbdK\\\xd4\xbcH/\xa1\xbc\xebq7=\x...


### Criando as Helper Functions que iremos utilizar

Aqui não tem segredo. Vamos desenhar as functions que irão nos apoiar nessa reta final. Já temos os embeddings, agora vamos guardar no Redis a entidade completa + embeddings, o que é uma boa prática.\

Aqui, vamos:
- Salvar os dados como HASH no Redis;
- Criar o RediSearch Index corretamente, explicando pro Redis o que é aquele embedding curioso que estamos adicionando no bolo.

In [ ]:
def load_dataframe(redis, df, key_prefix="tweet", id_column="id", pipe_size=100):
    records = df.to_dict(orient="records")
    pipe = redis.pipeline(transaction=False)
    i=1
    for record in tqdm(records):
        i=i+1
        key = f"{key_prefix}:{record[id_column]}"
        pipe.hset(key, mapping=record)
        if (i+1) % pipe_size == 0:
          res=pipe.execute()
    pipe.execute()

def create_redis_index(redis, idxname="tweet:idx"):
  try:
    redis.ft(idxname).dropindex()
  except:
    print("no index found")

  # Create an index
  indexDefinition = IndexDefinition(
      prefix=["tweet:"],
      index_type=IndexType.HASH,
  )

  redis.ft(idxname).create_index(
      (
          TextField("full_text", no_stem=False, sortable=False),
          VectorField("text_embedding", "HNSW", {  "TYPE": "FLOAT32",
                                                    "DIM": 384,
                                                    "DISTANCE_METRIC": "COSINE",
                                                  })
      ),
      definition=indexDefinition
  )



### Criando o index e carregando os dados para o Redis

In [ ]:
# Opcional - dar um reset brabo no Redis - vai perder tudo la
redis.flushdb()

# criando o Index
create_redis_index(redis)

# load data from Dataframe to Redis HASH
# carregando os dados do CSV, os tweets, para o Redis (como HASH, mas poderia ser JSON. Life's good.)
load_dataframe(redis,df,key_prefix="tweet", pipe_size=100)


no index found


  0%|          | 0/12420 [00:00<?, ?it/s]

In [ ]:
# De uma olhada em como o dado está ficando no seu Redis
# o text_embedding ta la, junto dos metadados do Tweet 1001, e sera a nossa impressao digital daqui pra frente
!redis-cli $REDIS_CONN hgetall "tweet:1001"

1) "text_embedding"
2) "\x16\x1e\x03:\xccb\xbd\xbcG\xb0\x94\xbc*K)<DU\x94;#>\x11\xbd\xaa3\r=\x03\x98\x7f=\x8cX\x88<qUR\xbd\xc5\xf6h\xbc\x85\xb5\xd2\xbb\x1c;\t\xbd\xf7m#<\xe7\xd4C=\x92\xc8\"\xbd\x16\xa3-\xbd/^\t\xbdn\x00\xfd\xbc\xbaGE\xbdk\xadZ\xbcV\xae=\xbdJ<\x99<\xedb\xce\xbc\xf1_\xab<BDT\xbciM\xd7\xbc \xf1\x82\xbb\x8d\xb0(=:v\x8a\xbc\xce\xa8M\xbd_,P=\xf7R\x86<\x99j\xfa<G?\xb5<\xcd[\xee\xbb\xcc\x8d\x90=\x13W8=Q\x81\x82\xbd\xa5\xfb&<\xf5-:=U\xe6\xe4\xbd\xe4\x1d\x11\xbc\xd3\xb7\xac\xbd\xdaY<\xbbH\xc9\xcc\xbb\xd89b\xbcI\xe3\x90=1\x9f\xa7=\xef7\x0f=J\xbb\xbb\xbbN\xad\x17=\xbe\xaeE\xbd\xdc\x8e\x90=s\xd7\x02\xbd\xd4\x01\x9f\xbc\xc7\xc3e;\x14\xa5\xd8\xbcm{f<\x9ex\xaf\xbd\xf6\xc4\x89\xbcQo\x03=oB\x8e\xbc\x88\x91G;\xcc\xe6\x97\xba\xbd\xc1\x9a=\x10\xf7B\xbd\xbe\r\xca<\xee\xf2G\xbb\xbe\x0e\x86<\xabr\x84<\xca\xf7^\xbd5\xc9\xbe\xbdpK5\xbc'\xe09\xbd\xc6\xd7==4y\xb3=\xad\x10\xb4\xbdr\xfc\x11\xbd\xfe\x86O\xbc\xc8o<\xbdf\x87\x80\xbc\x1d\xa9\"\xbdO) \xbdQ\xfb\x85\xba\xcay\x95=\xfc\xdf\xa6<\x03wF<\x0b\x

## Consulte o Redis, como um Banco de Dados primário agora!

[Always-on Streamlit app](https://antonum-redis-vss-streamlit-streamlit-app-p4z5th.streamlit.app/)


Vamos experimentar queries como:
“Oil”, “Oil Reserves”, “Fossil fuels”


Note que as buscas Full-Text que vão pelo "léxico apenas" são exauridas rapidamente... talvez abaixo do que você espera de uma Search Engine (como o ELK, por exemplo).

Entretanto, o Vector está com muitas (e boas) dimensões, pra gente trazer mais informação relevante para o nosso contexto aqui. Você que pode decidir depois como que o Vector Search está funcionando debaixo do capô.

In [ ]:
user_query="fossil fuels"
# use tbm "oil reserve", "fossil fuels"

In [ ]:
# usando a Full-Text contra o nosso Index
q = Query(user_query)\
  .return_fields("full_text")
res = redis.ft("tweet:idx").search(q)
if res.total==0:
  print("No matches found")
else:
  res_df = pd.DataFrame([t.__dict__ for t in res.docs ]).drop(columns=["payload"])
  display(res_df)

No matches found


In [ ]:
# a mesma parada, soh que melhor
# agora sim, usando um Vector Similarity contra o nosso Index
# notem que eu, malandramente, transformei a propria `user_query` em um vetor propio.
query_vector=text_to_embedding(user_query)
q = Query("*=>[KNN 10 @text_embedding $vector AS result_score]")\
                .return_fields("result_score","full_text")\
                .dialect(2)\
                .sort_by("result_score", True)
res = redis.ft("tweet:idx").search(q, query_params={"vector": query_vector})
#print(res)
res_df = pd.DataFrame([t.__dict__ for t in res.docs ]).drop(columns=["payload"])
res_df

,id,result_score,full_text
0,tweet:2907,0.610149264336,"Now in member chat: #Oil, #Energy sector, $XOM..."
1,tweet:2892,0.623246729374,"RT @philstockworld: Now in member chat: #Oil, ..."
2,tweet:5405,0.629442691803,https://t.co/3IJBXa5wuf Historic oil price plu...
3,tweet:5406,0.631321430206,Historic oil price plunge trashes sector's pro...
4,tweet:5676,0.632252693176,"I don't think the US can legally do that, and ..."
5,tweet:44,0.632612347603,https://t.co/9VjKMnpm7n\r\n\r\n#Cheniere Energ...
6,tweet:3568,0.632830858231,Top 200 energy stocks summary\r\n147 up\r\n48 ...
7,tweet:9402,0.639989078045,#OIL $SPY $CVS $XOM #companies $TSLA #TAX Scr...
8,tweet:5654,0.64328700304,..and oil still 25.74 LMAO &gt;&gt;&gt;NO DEM...
9,tweet:12459,0.643579661846,"@mintzmyer @tradewinds Fairness, you don’t fol..."
